## Importing files

In [30]:
import pandas as pd

gemma_baseline = pd.read_csv("../Results/QueryWise_gemma_resp_baseline.csv")
gemma_user_group_aware = pd.read_csv("../Results/QueryWise_gemma_resp_user_group_aware.csv")
gemma_user_need_aware = pd.read_csv("../Results/QueryWise_gemma_resp_user_need_aware.csv")

gpt_baseline = pd.read_csv("../Results/QueryWise_gpt_resp_baseline.csv")
gpt_user_group_aware = pd.read_csv("../Results/QueryWise_gpt_resp_user_group_aware.csv")
gpt_user_need_aware = pd.read_csv("../Results/QueryWise_gpt_resp_user_need_aware.csv")

In [31]:
gemma_baseline.head()

,QID,Task Sentiment,IAS,Resp_cleaned,positive,negative,joy,sadness,anger,fear
0,QGEN1,General,GEM,Il tornado è un'evento meteorologico potente e...,99.957770,0.042235,7.092738,0.451405,0.034272,92.421573
1,QGEN2,General,GEM,Il tornado è un'evento meteorologico potente e...,99.957770,0.042235,7.092738,0.451405,0.034272,92.421573
2,QGEN3,General,GEM,La piramide egizia è un monumento funerario di...,99.940920,0.059085,93.032539,2.686190,0.483058,3.798211
3,QGEN4,General,GEM,Le piramidi sono strutture monumentali che si ...,99.883336,0.116666,99.591297,0.231328,0.026861,0.150511
4,QGEN5,General,GEM,La piramide più alta dell'antico Egitto è la P...,99.964011,0.035983,99.835408,0.031344,0.030340,0.102904


## Significance testing

In [32]:
from scipy.stats import ttest_ind

emotions = ["positive", "negative", "joy", "sadness", "anger", "fear"]

def compare_logs(baseline_log, user_aware_log, need_aware_log, task_sent, IAS):
    baseline_log_task = baseline_log.loc[baseline_log["Task Sentiment"]==task_sent]
    user_aware_log_task = user_aware_log.loc[user_aware_log["Task Sentiment"]==task_sent]
    need_aware_log_task = need_aware_log.loc[need_aware_log["Task Sentiment"]==task_sent]

    results = []
    for emo in emotions:
        baseline_user_group_change = ttest_ind(baseline_log_task[emo], user_aware_log_task[emo]).pvalue
        baseline_user_need_change = ttest_ind(baseline_log_task[emo], need_aware_log_task[emo]).pvalue
        results.append([IAS, emo, "baseline_to_user_group", task_sent, baseline_user_group_change, baseline_user_group_change<0.05])
        results.append([IAS, emo, "baseline_to_user_need", task_sent, baseline_user_need_change, baseline_user_need_change<0.05])
    results_df = pd.DataFrame(results, columns=["IAS", "Emotion", "Comparison", "Task Sentiment", "p_val", "is_significant"])

    return results_df

In [33]:
task_sentiments = ["General", "Positive", "Negative"]

all_comparisons = []
for sent in task_sentiments:
    gemma_comparisons = compare_logs(gemma_baseline, gemma_user_group_aware, gemma_user_need_aware, sent, "GEM")
    gpt_comparisons = compare_logs(gpt_baseline, gpt_user_group_aware, gpt_user_need_aware, sent, "GPT")
    all_comparisons.append(gemma_comparisons)
    all_comparisons.append(gpt_comparisons)

all_comparisons_df = pd.concat(all_comparisons, ignore_index=True)
all_comparisons_df.to_csv("../Results/Stats.csv", index=False)
all_comparisons_df


,IAS,Emotion,Comparison,Task Sentiment,p_val,is_significant
0,GEM,positive,baseline_to_user_group,General,0.018429,True
1,GEM,positive,baseline_to_user_need,General,0.020889,True
2,GEM,negative,baseline_to_user_group,General,0.018429,True
3,GEM,negative,baseline_to_user_need,General,0.020889,True
4,GEM,joy,baseline_to_user_group,General,0.012244,True
...,...,...,...,...,...,...
67,GPT,sadness,baseline_to_user_need,Negative,0.000062,True
68,GPT,anger,baseline_to_user_group,Negative,0.523126,False
69,GPT,anger,baseline_to_user_need,Negative,0.283187,False
70,GPT,fear,baseline_to_user_group,Negative,0.093637,False
